In [ ]:
'''
Strategy: Weekly Buying based on Relative Strength [RS] and Hilega-Milega
Initial Screen Stocks from quarterly OHLC [Stocks Outperforming Index]
    - RS [compared with Nifty50] 
    - Trade Window: (RS > 0) to (RS < RS[-1])
    - Lookback 3 quarter
Weekly Screen from weekly OHLC
    - Buying HM-1 [Evaluates Price, Strength, Momentum and Volume]
    - WMA21 > WMA21[-1]
    - Positive RS and (RS[SMA5] > RS[-1][SMA5]) [Slope from Linear Regression should be tried]
Trade Parameter:
Entry Price
    - Type 1: Weekly Close
    - Type 2: 1/3 * [Entry Price - Stop Loss]
Stop Loss
    - Weekly Close < Swing Low Weekly     
Target
    - Type 1: 11 %
    - Type 2: 3 * [Entry Price - Stop Loss]
    
Input files: 
    - nse_total_list.csv
    - Datasets - quarterly and weekly OHLC (/raw_data/monthly_ohlc)
Output file: quarter_cup_handle_signals.csv
    - Stock_Name,Type,Entry_Type,Sig_Date,Cup_Date,Handle_Date,Buy_Price_1,Buy_Price_2
    - Type: [HM-1, HM-2, IN-1, IN-2]
    - Entry_Type : E1 [complete handle] - E2 [partial handle]
    - Buy_Price_1 : E1 trades
    - Buy_Price_2 : E2 trades

Idea:
HM signal   - Quarterly [HM-1, HM-2, IN-1, IN-2]
            - Entry_Price_1 [Entry_Price_2 from dataset]
Cup         - Monthly 
            - Init_1: Total Range < Entry_Price_2 
            - Sig_1: Close > Entry_Price_1
            - Entry_price_2 = low[Sig_1] (monthly close) 

Entry       - Type 1: Entry_Price_1
Target      - Type 1: 25 %


'''

'\nStrategy: Weekly Buying based on Relative Strength [RS] and Hilega-Milega\nInitial Screen Stocks from quarterly OHLC [Stocks Outperforming Index]\n    - RS [compared with Nifty50] \n    - Trade Window: (RS > 0) to (RS < RS[-1])\n    - Lookback 3 quarter\nWeekly Screen from weekly OHLC\n    - Buying HM-1 [Evaluates Price, Strength, Momentum and Volume]\n    - Positive RS and (RS > RS[-1]) [Slope from Linear Regression should be tried]\nTrade Parameter:\nEntry Price\n    - Type 1: Weekly Close\n    - Type 2: 1/3 * [Entry Price - Stop Loss]\nStop Loss\n    - Weekly Close < Swing Low Weekly     \nTarget\n    - Type 1: 11 %\n    - Type 2: 3 * [Entry Price - Stop Loss]\n    \nInput files: \n    - nse_total_list.csv\n    - Datasets - quarterly and weekly OHLC (/raw_data/monthly_ohlc)\nOutput file: quarter_cup_handle_signals.csv\n    - Stock_Name,Type,Entry_Type,Sig_Date,Cup_Date,Handle_Date,Buy_Price_1,Buy_Price_2\n    - Type: [HM-1, HM-2, IN-1, IN-2]\n    - Entry_Type : E1 [complete handl

In [37]:
import sys
main_path = r'/Gokul/PYTHON-VS' 
sys.path.append(main_path) 

from scripts_Algo_GP import * 
from scipy.stats import linregress
import tradingview_indicators as ta
from pprint import pprint
import csv, math
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [38]:
def rs_NIFTY(df,df_index,rs_length):
    # compute index s_ratio   
    df_index['i_ratio'] = df_index['close'] / df_index['close'].shift(rs_length)
    # compute return of stock
    df['s_ratio'] = df['close'] / df['close'].shift(rs_length)
    df['i_ratio'] = df_index['i_ratio']
    # Compute RS
    df['RS'] = ((df['s_ratio']/df['i_ratio'])-1).round(2)
    df = df.drop(columns=['s_ratio','i_ratio'])
    return df
#((152.96 /130.05)/(23644/22326))-1

In [ ]:
def main():
    '''------ Load Input Files ---------'''
    file_name = f"{main_path}/nse_list_2025_01.csv"
    file = open(file_name, "r")
    list_stock = list(csv.reader(file, delimiter=","))
    file.close()
    #print(list_stock)
    '''------ Input Parameters ---------'''
    rs_qtr_length = 3
    '''------ Load Index ---------'''
    df_qtr_index = pd.read_csv(f"{main_path}/raw_data/quarter_ohlc/quarter_NIFTY.csv")
    #df_week_index = pd.read_csv(f"{main_path}/raw_data/weekly_ohlc/weekly_NIFTY.csv")
    #print(df_qtr_index.tail(20))
    '''------ Loop for each Stock---------'''
    for stock_id in list_stock[2:3]: 
        id_stock = stock_id[0]
        id_index= stock_id[1]
        qtr_file = f"{main_path}/raw_data/quarter_ohlc/quarter_{id_stock[4:-3]}.csv"
        week_file = f"{main_path}/raw_data/weekly_ohlc/weekly_{id_stock[4:-3]}.csv"
        print(id_stock)
        print(qtr_file)
        #print(week_file)

        '''------ Initial Screen ---------'''
        df_qtr = pd.read_csv(qtr_file)
        df_qtr['datetime'] = pd.to_datetime(df_qtr['datetime']).dt.strftime('%Y-%m-%d')
        # Match the Size of the two DFs and reset Index number
        df_index = df_qtr_index.iloc[-len(df_qtr):].reset_index(drop=True)
        # Compute Relative Strength with Nifty50
        df_qtr = rs_NIFTY(df_qtr,df_index,rs_qtr_length)
        df_qtr = df_qtr[['datetime','RS']]
        df_qtr['start_RS'] = df_qtr['end_RS'] = 0  
        # Tradeable Intervals 
        mask_start= ((df_qtr['RS'] > 0) & (df_qtr['RS'].shift(1) <= 0)) \
            | ((df_qtr['RS'] > 0) & (df_qtr['RS'].shift(1) < df_qtr['RS']) \
            & (df_qtr['RS'].shift(1) < df_qtr['RS'].shift(2)))   
        df_qtr.loc[mask_start.shift(1).astype(bool).fillna(False), 'start_RS'] = 1
        df_qtr.at[0,'start_RS'] = 0
        mask_end = (df_qtr['RS'].shift(1) > df_qtr['RS']) \
            & (df_qtr['RS'].shift(1) > df_qtr['RS'].shift(2)) \
            & (df_qtr['RS'].shift(1) > 0)
        df_qtr.loc[mask_end, 'end_RS'] = -1
        # Add trading interval to df
        df_qtr.loc[(df_qtr['start_RS'] == 1), 'start_interval'] = df_qtr.loc[(df_qtr['start_RS'] == 1), 'datetime']
        df_qtr.loc[(df_qtr['end_RS'] == -1), 'end_interval'] = pd.to_datetime(df_qtr.loc[(df_qtr['end_RS'] == -1), 'datetime']) + pd.offsets.QuarterEnd(0)
        df_qtr['end_interval'] = df_qtr['end_interval'].dt.strftime('%Y-%m-%d')
        # Create an array of tradeable interval Dates
        start_array = df_qtr['start_interval'].dropna().tolist()
        end_array = df_qtr['end_interval'].dropna().tolist()
        print(start_array)
        print(end_array)
        
        '''------ Weekly Screen ---------'''



if __name__ == '__main__':
    main()


NSE:KPITTECH-EQ
/Gokul/PYTHON-VS/raw_data/quarter_ohlc/quarter_KPITTECH.csv
      datetime    RS  start_RS  end_RS
0   2019-04-01   NaN         0       0
1   2019-07-01   NaN         0       0
2   2019-10-01   NaN         0       0
3   2020-01-01 -0.50         0       0
4   2020-04-01 -0.26         0       0
5   2020-07-01  0.41         0       0
6   2020-10-01  1.48         1       0
7   2021-01-01  0.98         0       0
8   2021-04-01  0.52         0       0
9   2021-07-01  0.91         0       0
10  2021-10-01  1.92         1       0
11  2022-01-01  1.08         0       0
12  2022-04-01  0.69         0       0
13  2022-07-01  0.09         0       0
14  2022-10-01  0.13         0       0
15  2023-01-01  0.62         1       0
16  2023-04-01  0.47         0       0
17  2023-07-01  0.51         0       0
18  2023-10-01  0.31         1       0
19  2024-01-01  0.17         0       0
20  2024-04-01  0.16         0       0
21  2024-07-01 -0.09         0       0
22  2024-10-01 -0.07       

In [40]:
#((67.37/ )/ (1263.55-1528.45))-1
#((239.68/226.82)/ (5647.50/6134.05))-1
#((152.96 /130.05)/(23644/22326))-1
((121.29 /119.85)/(25810/21731))-1




-0.14792334858712863